In [ ]:
# Jupyter cell (index 0)
# Read Lorentz_Part_*.csv (columns: id,rank,X,Y), plot scatter for each file and create a video.
# Saves lorentz_animation.mp4 (falls back to GIF if ffmpeg not available) and displays it.

import glob
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.colors import Normalize
from IPython.display import Video, display
from tqdm import tqdm

pattern = "Lorentz_Part_*.csv"
files = sorted(glob.glob(pattern))
if not files:
    raise FileNotFoundError(f"No files matching pattern: {pattern}")

# Read all frames with a progress bar
frames = []
for f in tqdm(files, desc="Reading CSV frames"):
    df = pd.read_csv(f)
    frames.append(df)

# Ensure expected columns exist
required = {"id", "rank", "X", "Y"}
for i, df in enumerate(frames):
    if not required.issubset(df.columns):
        raise ValueError(f"File {files[i]} is missing required columns. Found: {list(df.columns)}")

# Compute global axis limits
all_x = np.concatenate([df["X"].to_numpy() for df in frames])
all_y = np.concatenate([df["Y"].to_numpy() for df in frames])
pad_x = (all_x.max() - all_x.min()) * 0.02 if np.ptp(all_x) != 0 else 1.0
pad_y = (all_y.max() - all_y.min()) * 0.02 if np.ptp(all_y) != 0 else 1.0
xlim = (all_x.min() - pad_x, all_x.max() + pad_x)
ylim = (all_y.min() - pad_y, all_y.max() + pad_y)

# Color by rank with a stable normalization across frames
all_ranks = np.concatenate([df["rank"].to_numpy() for df in frames])
norm = Normalize(vmin=float(np.min(all_ranks)), vmax=float(np.max(all_ranks)))

fig, ax = plt.subplots(figsize=(6, 6))
sc = ax.scatter([], [], s=8, c=[], cmap="viridis", norm=norm, edgecolors="none")
ax.set_xlim(*xlim)
ax.set_ylim(*ylim)
ax.set_xlabel("X")
ax.set_ylabel("Y")
cb = plt.colorbar(sc, ax=ax, fraction=0.046, pad=0.04)
cb.set_label("rank")
title = ax.set_title("")

def init():
    sc.set_offsets(np.empty((0, 2)))
    sc.set_array(np.array([]))
    title.set_text("")
    return sc, title

# tqdm progress bar will be updated by the animation update() callback
pbar = tqdm(total=len(frames), desc="Rendering frames", leave=False)
def update(i):
    df = frames[i]
    coords = df[["X", "Y"]].to_numpy()
    sc.set_offsets(coords)
    sc.set_array(df["rank"].to_numpy())
    title.set_text(f"Frame {i+1}/{len(frames)}  —  {Path(files[i]).name}")
    pbar.update(1)
    return sc, title

anim = animation.FuncAnimation(
    fig, update, frames=len(frames), init_func=init, blit=True, interval=100, repeat=False
)

# Try to save as MP4 via ffmpeg, fallback to GIF via Pillow
out_mp4 = "lorentz_animation.mp4"
out_gif = "lorentz_animation.gif"
fps = 10

saved_path = None
try:
    Writer = animation.FFMpegWriter
    writer = Writer(fps=fps, metadata=dict(artist="generated"), bitrate=2000)
    anim.save(out_mp4, writer=writer, dpi=150)
    saved_path = out_mp4
except Exception:
    try:
        anim.save(out_gif, writer="pillow", fps=fps, dpi=150)
        saved_path = out_gif
    except Exception as e:
        pbar.close()
        plt.close(fig)
        raise RuntimeError(
            "Failed to save animation as MP4 or GIF. Ensure ffmpeg or pillow is available."
        ) from e
finally:
    pbar.close()

plt.close(fig)  # avoid duplicate static plot display

# Display the movie inline
display(Video(saved_path, embed=True))
print(f"Saved animation to: {saved_path}")